In [1]:
!pip.exe install google-cloud-aiplatform

In [1]:
teste = [
    {   # Hipotecas / Empréstimos (Mortgages / Loans)
        "descricao_reclamacao_processed": 
        "Fiz um pedido de empréstimo mas os juros aplicados são muito superiores aos que foram inicialmente informados."
    },
    {   # Cartão de crédito / Cartão pré-pago (Credit Card / Prepaid Card)
        "descricao_reclamacao_processed": 
        "Recebi cobranças não autorizadas no meu cartão de crédito, preciso de uma solução urgente."
    },
    {   # Serviços de conta bancária (Banking Account Services)
        "descricao_reclamacao_processed": 
        "Tentei acessar minha conta pelo aplicativo do banco e recebo uma mensagem de erro, não consigo visualizar meu saldo."
    }
]

In [2]:
from google.cloud import aiplatform
from google.oauth2 import service_account

In [5]:
# Path to your service account key file
SAC_KEY_FP = "topic-modelling-fiap-417601-432750c09ce7.json"
SAC_UNIQUEID = "115532493627132210234"
ENDPOINT_ID = "6120385296526737408"

# Creating credentials using the key file
#my_credentials = service_account.Credentials.from_service_account_file(
#    SAC_KEY_FP
#)

In [3]:
# New way to authenticate without exposing the credentials
import base64
import json

# Load the encoded string from the new text file
with open('encoded-auth.txt', 'r') as encoded_file:
    encoded_string = encoded_file.read()

# Decode the string back to JSON
decoded_bytes = base64.b64decode(encoded_string)
decoded_json = json.loads(decoded_bytes.decode('utf-8'))

# Create credentials from the decoded JSON
my_credentials = service_account.Credentials.from_service_account_info(decoded_json)

In [6]:
aiplatform.init(
    # your Google Cloud Project ID or number
    # environment default used is not set
    project='topic-modelling-fiap-417601',

    # the Vertex AI region you will use
    # defaults to us-central1
    location='us-central1',

    # Google Cloud Storage bucket in same region as location
    # used to stage artifacts
    staging_bucket='gs://tm-fiap-bucket',

    # custom google.auth.credentials.Credentials
    # environment default credentials used if not set
    credentials=my_credentials,

    # customer managed encryption key resource name
    # will be applied to all Vertex AI resources if set
    encryption_spec_key_name=SAC_UNIQUEID
)

In [7]:
endpoint = aiplatform.Endpoint(ENDPOINT_ID)

In [8]:
# Make the prediction
response = endpoint.predict(instances=teste)

# Process the response
for prediction in response.predictions:
    
    # Extract classes and scores from the prediction
    predicted_classes = prediction['classes']
    predicted_scores = prediction['scores']

    # Print each class with its corresponding score
    print("Prediction results:")
    for predicted_class, score in zip(predicted_classes, predicted_scores):
        print(f"{predicted_class}: {score:.2f}")
    print("-------------------")

Prediction results:
Serviços de conta bancária: 0.02
Cartão de crédito / Cartão pré-pago: 0.03
Roubo / Relatório de disputa: 0.03
Hipotecas / Empréstimos: 0.90
Outros: 0.02
-------------------
Prediction results:
Serviços de conta bancária: 0.07
Cartão de crédito / Cartão pré-pago: 0.58
Roubo / Relatório de disputa: 0.25
Hipotecas / Empréstimos: 0.06
Outros: 0.05
-------------------
Prediction results:
Serviços de conta bancária: 0.86
Cartão de crédito / Cartão pré-pago: 0.05
Roubo / Relatório de disputa: 0.03
Hipotecas / Empréstimos: 0.03
Outros: 0.02
-------------------
